# Extreme Natural events: Cyclones

## Data Extraction from EONET NASA API

In [38]:
import pandas as pd
import numpy as np
import re
import math
import requests
import json
import matplotlib
from pandas import ExcelFile
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
from datetime import datetime
import plotly.express
import reverse_geocoder as rg
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots


The natural events in the EONET's API are divided in two primarily groups according with their status:

* ```'status' = 'open'``` means the natural event has not ended.

* ```'status' = 'closed'``` means the natural event has finished. When this happen we will have as well an aditional field labeled as closed.

## 1. Data extraction and wrangling

### Extracting data from the EONET's API

In [2]:
#For extracting the category of each natural event

def extract_cat(lista):
    """
    Takes a list of dictionaries and return the value of the first element
    of the list for the key 'title'
    """
    return lista[0]["title"]


# Extracts the date of each natural event
def extract_date(lista):
    """
    Takes a list of dictionaries and return the value of the first element
    of the list for the key 'date'
    """
    return lista[0]["date"]

# Extracts the type of Geojson coordinates of each natural event

def extract_geom(lista):
    """
    Takes a list of dictionaries and removes the key 'date' for the first 
    dictionary
    """
    del lista[0]["date"]
    return lista[0]
    
base = "https://eonet.sci.gsfc.nasa.gov/api/v2.1/events?status=" 

def get_events(st):
    
    """
    Arguments: "open" or "closed"
    Returns a the data set of the events with status "st" 
    which happened since 1980-01-10 until teh current date.
    
    """
    url = base + f"{st}"
    result = requests.get(url)
    
    df_0 = pd.DataFrame(result.json()["events"], columns = ["id", "title", "categories", "description", "sources", "geometries", "closed"])
    df_0.categories = df_0.categories.apply(extract_cat) # Extract category from the original dictionary
    df_0["date"] = df_0.geometries.apply(extract_date)
    df_0["geometries"] = df_0.geometries.apply(extract_geom)
    df_0 = df_0.drop(columns = ["sources"])
    return df_0[['id', 'title', 'categories', 'description', 'date', 'closed','geometries']]


In [3]:
df_op = get_events("open")
df_cl = get_events("closed")

In [4]:
df_ev = (pd
        .concat([df_op, df_cl], ignore_index=True)
        .sort_values("date", ascending = False)
        .reset_index(drop=True))

df_ev.dtypes

id             object
title          object
categories     object
description    object
date           object
closed         object
geometries     object
dtype: object

In [5]:
df_ev.head()

,id,title,categories,description,date,closed,geometries
0,EONET_4611,"Wildfires - Carahue, Chile",Wildfires,,2020-03-06T14:58:00Z,NaN,"{'type': 'Point', 'coordinates': [-73.38841511..."
1,EONET_4609,"Wildfire - Traiguen, Chile",Wildfires,,2020-03-05T15:39:00Z,NaN,"{'type': 'Point', 'coordinates': [-72.67187, -..."
2,EONET_4608,"Wildfire - Galvarino, Chile",Wildfires,,2020-03-05T15:37:00Z,NaN,"{'type': 'Point', 'coordinates': [-73.39149242..."
3,EONET_4610,"Wildfire - Quillon, Chile",Wildfires,,2020-03-05T15:36:00Z,NaN,"{'type': 'Point', 'coordinates': [-72.46852, -..."
4,EONET_4605,"Wildfire - Riverside [Mann Fire], California",Wildfires,,2020-03-03T14:31:00Z,NaN,"{'type': 'Point', 'coordinates': [-117.55095, ..."


In [6]:
df_ev.categories.value_counts()

Severe Storms           1822
Wildfires               1533
Volcanoes                863
Floods                   116
Sea and Lake Ice          60
Water Color               17
Dust and Haze             17
Earthquakes               15
Temperature Extremes      14
Drought                    7
Manmade                    5
Snow                       3
Landslides                 2
Name: categories, dtype: int64

In [7]:
df_ev["date"]= pd.to_datetime(df_ev['date'],infer_datetime_format = True).dt.date
df_ev["closed"]= pd.to_datetime(df_ev['closed'],infer_datetime_format = True).dt.date

In [8]:
df_ev["date"]= pd.to_datetime(df_ev['date'],infer_datetime_format = True)
df_ev["closed"]= pd.to_datetime(df_ev['closed'],infer_datetime_format = True)

In [9]:
df_ev.dtypes

id                     object
title                  object
categories             object
description            object
date           datetime64[ns]
closed         datetime64[ns]
geometries             object
dtype: object

In [10]:
df_ev.sort_values("date")

,id,title,categories,description,date,closed,geometries
4473,EONET_713,"Tupungatito Volcano, Chile-Argentina",Volcanoes,,1980-01-10,1980-01-11,"{'type': 'Point', 'coordinates': [-69.797, -33..."
4472,EONET_936,"Nyamuragira Volcano, DR Congo",Volcanoes,,1980-01-30,1980-02-24,"{'type': 'Point', 'coordinates': [29.2, -1.408]}"
4471,EONET_533,"Krakatau Volcano, Indonesia",Volcanoes,,1980-03-15,1980-12-31,"{'type': 'Point', 'coordinates': [105.423, -6...."
4470,EONET_1043,"St. Helens Volcano, Washington",Volcanoes,,1980-03-27,1986-10-28,"{'type': 'Point', 'coordinates': [-122.18, 46.2]}"
4469,EONET_886,"Lopevi Volcano, Vanuatu",Volcanoes,,1980-04-15,1980-08-20,"{'type': 'Point', 'coordinates': [168.346, -16..."
...,...,...,...,...,...,...,...
4,EONET_4605,"Wildfire - Riverside [Mann Fire], California",Wildfires,,2020-03-03,NaT,"{'type': 'Point', 'coordinates': [-117.55095, ..."
3,EONET_4610,"Wildfire - Quillon, Chile",Wildfires,,2020-03-05,NaT,"{'type': 'Point', 'coordinates': [-72.46852, -..."
2,EONET_4608,"Wildfire - Galvarino, Chile",Wildfires,,2020-03-05,NaT,"{'type': 'Point', 'coordinates': [-73.39149242..."
1,EONET_4609,"Wildfire - Traiguen, Chile",Wildfires,,2020-03-05,NaT,"{'type': 'Point', 'coordinates': [-72.67187, -..."


In [11]:
storms = df_ev[df_ev.categories== "Severe Storms"]

In [12]:
storms.head()

,id,title,categories,description,date,closed,geometries
13,EONET_4596,Tropical Cyclone Ferdinand,Severe Storms,,2020-02-23,2020-03-05,"{'type': 'Point', 'coordinates': [118.7, -13.4]}"
14,EONET_4595,Tropical Cyclone Esther,Severe Storms,,2020-02-23,2020-03-04,"{'type': 'Point', 'coordinates': [139.5, -15.6]}"
18,EONET_4597,Tropical Cyclone Wasi,Severe Storms,,2020-02-21,2020-02-27,"{'type': 'Point', 'coordinates': [-175.8, -12.4]}"
20,EONET_4590,Tropical Cyclone Vicky,Severe Storms,,2020-02-20,2020-02-26,"{'type': 'Point', 'coordinates': [-171.4, -14.4]}"
23,EONET_4586,Tropical Cyclone Gabekile,Severe Storms,,2020-02-15,2020-02-23,"{'type': 'Point', 'coordinates': [74.7, -16.0]}"


In [13]:
cyclone= storms[storms.title.str.contains("Cyclone")].copy()

In [14]:
other_storms= storms[~storms.title.str.contains("Cyclone")]

In [15]:
cyclone.head()

,id,title,categories,description,date,closed,geometries
13,EONET_4596,Tropical Cyclone Ferdinand,Severe Storms,,2020-02-23,2020-03-05,"{'type': 'Point', 'coordinates': [118.7, -13.4]}"
14,EONET_4595,Tropical Cyclone Esther,Severe Storms,,2020-02-23,2020-03-04,"{'type': 'Point', 'coordinates': [139.5, -15.6]}"
18,EONET_4597,Tropical Cyclone Wasi,Severe Storms,,2020-02-21,2020-02-27,"{'type': 'Point', 'coordinates': [-175.8, -12.4]}"
20,EONET_4590,Tropical Cyclone Vicky,Severe Storms,,2020-02-20,2020-02-26,"{'type': 'Point', 'coordinates': [-171.4, -14.4]}"
23,EONET_4586,Tropical Cyclone Gabekile,Severe Storms,,2020-02-15,2020-02-23,"{'type': 'Point', 'coordinates': [74.7, -16.0]}"


In [16]:
list_cyclone= list(cyclone.title.value_counts().index)

As observed, we have geometries but explicit cities or countries in which the extreme events took place. For that reason we exract the latitud and longitud from ``geometries`` in order to use ``reverse_geocoder`` module:

In [17]:
cyclone["lat"]= cyclone.geometries.apply(lambda x: x["coordinates"][1]).copy()
cyclone["lon"]= cyclone["geometries"].apply(lambda x: x["coordinates"][0]).copy()

In [18]:
cyclone.head()

,id,title,categories,description,date,closed,geometries,lat,lon
13,EONET_4596,Tropical Cyclone Ferdinand,Severe Storms,,2020-02-23,2020-03-05,"{'type': 'Point', 'coordinates': [118.7, -13.4]}",-13.4,118.7
14,EONET_4595,Tropical Cyclone Esther,Severe Storms,,2020-02-23,2020-03-04,"{'type': 'Point', 'coordinates': [139.5, -15.6]}",-15.6,139.5
18,EONET_4597,Tropical Cyclone Wasi,Severe Storms,,2020-02-21,2020-02-27,"{'type': 'Point', 'coordinates': [-175.8, -12.4]}",-12.4,-175.8
20,EONET_4590,Tropical Cyclone Vicky,Severe Storms,,2020-02-20,2020-02-26,"{'type': 'Point', 'coordinates': [-171.4, -14.4]}",-14.4,-171.4
23,EONET_4586,Tropical Cyclone Gabekile,Severe Storms,,2020-02-15,2020-02-23,"{'type': 'Point', 'coordinates': [74.7, -16.0]}",-16.0,74.7


In [19]:
cyclone_p=  cyclone[["lat", "lon"]].copy()

In [20]:
cyclone["coordinates"]= cyclone_p.values.tolist()

In [21]:
cyclone.head()

,id,title,categories,description,date,closed,geometries,lat,lon,coordinates
13,EONET_4596,Tropical Cyclone Ferdinand,Severe Storms,,2020-02-23,2020-03-05,"{'type': 'Point', 'coordinates': [118.7, -13.4]}",-13.4,118.7,"[-13.4, 118.7]"
14,EONET_4595,Tropical Cyclone Esther,Severe Storms,,2020-02-23,2020-03-04,"{'type': 'Point', 'coordinates': [139.5, -15.6]}",-15.6,139.5,"[-15.6, 139.5]"
18,EONET_4597,Tropical Cyclone Wasi,Severe Storms,,2020-02-21,2020-02-27,"{'type': 'Point', 'coordinates': [-175.8, -12.4]}",-12.4,-175.8,"[-12.4, -175.8]"
20,EONET_4590,Tropical Cyclone Vicky,Severe Storms,,2020-02-20,2020-02-26,"{'type': 'Point', 'coordinates': [-171.4, -14.4]}",-14.4,-171.4,"[-14.4, -171.4]"
23,EONET_4586,Tropical Cyclone Gabekile,Severe Storms,,2020-02-15,2020-02-23,"{'type': 'Point', 'coordinates': [74.7, -16.0]}",-16.0,74.7,"[-16.0, 74.7]"


In [22]:
cyclone["coordinates"]= cyclone.coordinates.apply(lambda x: (x[0], x[1]))
cyclone["location"]= cyclone.coordinates.apply(lambda x: rg.search(x)[0]["name"])
cyclone["admin"]= cyclone.coordinates.apply(lambda x: rg.search(x)[0]["admin1"])
cyclone["code"]= cyclone.coordinates.apply(lambda x: rg.search(x)[0]["cc"])


Loading formatted geocoded file...


In [23]:
cyclone.head()

,id,title,categories,description,date,closed,geometries,lat,lon,coordinates,location,admin,code
13,EONET_4596,Tropical Cyclone Ferdinand,Severe Storms,,2020-02-23,2020-03-05,"{'type': 'Point', 'coordinates': [118.7, -13.4]}",-13.4,118.7,"(-13.4, 118.7)",Lailunggi,East Nusa Tenggara,ID
14,EONET_4595,Tropical Cyclone Esther,Severe Storms,,2020-02-23,2020-03-04,"{'type': 'Point', 'coordinates': [139.5, -15.6]}",-15.6,139.5,"(-15.6, 139.5)",Alyangula,Northern Territory,AU
18,EONET_4597,Tropical Cyclone Wasi,Severe Storms,,2020-02-21,2020-02-27,"{'type': 'Point', 'coordinates': [-175.8, -12.4]}",-12.4,-175.8,"(-12.4, -175.8)",Mata-Utu,Circonscription d'Uvea,WF
20,EONET_4590,Tropical Cyclone Vicky,Severe Storms,,2020-02-20,2020-02-26,"{'type': 'Point', 'coordinates': [-171.4, -14.4]}",-14.4,-171.4,"(-14.4, -171.4)",Matavai,,WS
23,EONET_4586,Tropical Cyclone Gabekile,Severe Storms,,2020-02-15,2020-02-23,"{'type': 'Point', 'coordinates': [74.7, -16.0]}",-16.0,74.7,"(-16.0, 74.7)",Port Mathurin,Rodrigues,MU


In [28]:
codes= pd.read_excel("/Users/Fabi/Documents/GitHub/wfpdisrel/country_codes.xlsx")

In [29]:
codes.head()

,Country,ISO2,ISO3,Numeric
0,Afghanistan,AF,AFG,4
1,Albania,AL,ALB,8
2,Algeria,DZ,DZA,12
3,American Samoa,AS,ASM,16
4,Andorra,AD,AND,20


In [33]:
cyclone= cyclone.merge(codes[["ISO2", "ISO3", "Country"]], left_on= "code", right_on= "ISO2", how= "left").drop(columns= "code")

In [34]:
cyclone

,id,title,categories,description,date,closed,geometries,lat,lon,coordinates,location,admin,ISO2,ISO3,Country
0,EONET_4596,Tropical Cyclone Ferdinand,Severe Storms,,2020-02-23,2020-03-05,"{'type': 'Point', 'coordinates': [118.7, -13.4]}",-13.4,118.7,"(-13.4, 118.7)",Lailunggi,East Nusa Tenggara,ID,IDN,Indonesia
1,EONET_4595,Tropical Cyclone Esther,Severe Storms,,2020-02-23,2020-03-04,"{'type': 'Point', 'coordinates': [139.5, -15.6]}",-15.6,139.5,"(-15.6, 139.5)",Alyangula,Northern Territory,AU,AUS,Australia
2,EONET_4597,Tropical Cyclone Wasi,Severe Storms,,2020-02-21,2020-02-27,"{'type': 'Point', 'coordinates': [-175.8, -12.4]}",-12.4,-175.8,"(-12.4, -175.8)",Mata-Utu,Circonscription d'Uvea,WF,WLF,Wallis and Futuna
3,EONET_4590,Tropical Cyclone Vicky,Severe Storms,,2020-02-20,2020-02-26,"{'type': 'Point', 'coordinates': [-171.4, -14.4]}",-14.4,-171.4,"(-14.4, -171.4)",Matavai,,WS,WSM,Samoa
4,EONET_4586,Tropical Cyclone Gabekile,Severe Storms,,2020-02-15,2020-02-23,"{'type': 'Point', 'coordinates': [74.7, -16.0]}",-16.0,74.7,"(-16.0, 74.7)",Port Mathurin,Rodrigues,MU,MUS,Mauritius


In [37]:
cyclone.Country.value_counts()

Australia                         51
Mauritius                         38
Cocos (Keeling) Islands (the)     31
India                             27
Vanuatu                           26
Maldives                          25
Indonesia                         20
Wallis and Futuna                 19
Madagascar                        17
Solomon Islands                   17
Fiji                              15
Papua New Guinea                  12
American Samoa                    12
Tonga                              7
Christmas Island                   7
Seychelles                         6
Mozambique                         6
New Caledonia                      4
Tokelau                            4
Sri Lanka                          3
Yemen                              3
Niue                               2
Timor-Leste                        2
French Polynesia                   2
Mexico                             2
Samoa                              2
Viet Nam                           2
T

In [40]:
cyclone.to_json("cyclone.json")

In [36]:
fig= px.choropleth(co2_plot, 
                   locations="ISO3", 
                   color="CO2 emissions", 
                   hover_name="Country",
                   animation_frame="Year",
                   animation_group="ISO3",
                   #marker_line_color='white',
                   projection="robinson",
                   #color_continuous_scale = lisbon
                   color_continuous_scale= oslorev#px.colors.sequential.Inferno[::-1],
                   #range_color=[47,10877.217944]
                   )
                

                   #range_color=[47,10877.217944])

fig.update_layout(
    title_text = 'CO2 emissions (Mton)'
    #marker_line_color='white'# Create a Title
    #geo_scope='usa',  # Plot only the USA instead of globe
)
fig.show() 


Series([], dtype: float64)